# Example1.py

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='challenge')

22/04/08 11:00:33 WARN Utils: Your hostname, Dominique resolves to a loopback address: 127.0.1.1; using 172.22.70.171 instead (on interface eth0)
22/04/08 11:00:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/08 11:00:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SparkSession

In [4]:
ss = SparkSession.builder.getOrCreate()

In [5]:
dft = ss.read.csv('./node_information.csv') # ici, on lit un fichier qui peut être sur le cluster (auquel cas il faut mettre l'adresse du cluster)

In [6]:
# load articles
df_articles = dft.toDF('id', 'year', 'topics', 'authors', 'empty', 'abstract')
print(df_articles.head(1))

[Row(id='1001', year='2000', topics='compactification geometry and duality', authors='Paul S. Aspinwall', empty=None, abstract='these are notes based on lectures given at tasi99 we review the geometry of the moduli space of n 2 theories in four dimensions from the point of view of superstring compactification the cases of a type iia or type iib string compactified on a calabi-yau threefold and the heterotic string compactified on k3xt2 are each considered in detail we pay specific attention to the differences between n 2 theories and n 2 theories the moduli spaces of vector multiplets and the moduli spaces of hypermultiplets are reviewed in the case of hypermultiplets this review is limited by the poor state of our current understanding some peculiarities such as mixed instantons and the non-existence of a universal hypermultiplet are discussed')]


In [7]:
# load edges
df_links = ss.read.csv('./training_set.txt', sep=" ").toDF('source', 'target', 'class')
print("links dataframe")
print(df_links.head(1))
print("############################################")

links dataframe
[Row(source='9510123', target='9502114', class='1')]
############################################


In [8]:
# Creation of a new column
from pyspark.sql.functions import lower, col

df_articles = df_articles.withColumn('title_lower', lower(col('topics')))
print("create new column")
print(df_articles.head(1))
print("################")

create new column
[Row(id='1001', year='2000', topics='compactification geometry and duality', authors='Paul S. Aspinwall', empty=None, abstract='these are notes based on lectures given at tasi99 we review the geometry of the moduli space of n 2 theories in four dimensions from the point of view of superstring compactification the cases of a type iia or type iib string compactified on a calabi-yau threefold and the heterotic string compactified on k3xt2 are each considered in detail we pay specific attention to the differences between n 2 theories and n 2 theories the moduli spaces of vector multiplets and the moduli spaces of hypermultiplets are reviewed in the case of hypermultiplets this review is limited by the poor state of our current understanding some peculiarities such as mixed instantons and the non-existence of a universal hypermultiplet are discussed', title_lower='compactification geometry and duality')]
################


In [9]:
# do a join
inner_join_example = df_articles.join(df_links, df_articles.id == df_links.source)
print("join with python function")
print(inner_join_example.head(1))
print("################")

join with python function


[Row(id='10010', year='2000', topics='supersymmetry and f-theory realization of the deformed conifold with', authors='Steven S. Gubser (Princeton University)', empty=None, abstract='three-form flux it is shown that the deformed conifold solution with three-form flux found by klebanov and strassler is supersymmetric and that it admits a simple f-theory description in terms of a direct product of the deformed conifold and a torus some general remarks on ramond-ramond backgrounds and warped compactifications are included', title_lower='supersymmetry and f-theory realization of the deformed conifold with', source='10010', target='107263', class='0')]
################


In [10]:
inner_join_example = inner_join_example.drop(inner_join_example.topics)
print("drop a column")
print(inner_join_example.head(1))
print("################")

drop a column


[Row(id='10010', year='2000', authors='Steven S. Gubser (Princeton University)', empty=None, abstract='three-form flux it is shown that the deformed conifold solution with three-form flux found by klebanov and strassler is supersymmetric and that it admits a simple f-theory description in terms of a direct product of the deformed conifold and a torus some general remarks on ramond-ramond backgrounds and warped compactifications are included', title_lower='supersymmetry and f-theory realization of the deformed conifold with', source='10010', target='107263', class='0')]
################


In [11]:
#We can do both and more complex stuff with SQL as well
#But first we need tables to refer to 

df_articles.createOrReplaceTempView("text")
df_links.createOrReplaceTempView("links")
print("simple select with SQL")
print(ss.sql("Select source,target,class from links where source like '%par%'" ).head(1))
print("################")


simple select with SQL
[]
################


In [12]:
#Count how many elements per class
print("count classes")
print(ss.sql("Select class,count(*) from links group by class" ).show())
print("################")

count classes
+-----+--------+
|class|count(1)|
+-----+--------+
|    0|  280382|
|    1|  335130|
+-----+--------+

None
################


In [14]:
#Join and select some of the columns
inner_join_example2=ss.sql("""Select source,target, text1.abstract as txt1,class 
								from links 
								inner join text as text1 on text1.title_lower==source""")
print("join and column selection with SQL")
print(inner_join_example2.head(1))
print("################")

join and column selection with SQL
[]
################


# Example2.py

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import lower, col
import numpy as np
from pyspark.ml.linalg import SparseVector, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType

In [16]:
spark = SparkSession.builder.appName('example').getOrCreate()
sc = spark.sparkContext

In [39]:
df_articles = spark.read.csv("./node_information.csv",sep=",").toDF("ID","Year","Intro","Authors","Empty","Abstract")
df_links = spark.read.csv("./training_set.txt",sep=" ").toDF("source","target","class")
df_articles = df_articles.withColumn("title_lower",lower(col("Intro")))

In [40]:
df_articles.createOrReplaceTempView("text")
df_links.createOrReplaceTempView("links")

In [47]:
inner_join=spark.sql("""Select source,target, text1.title_lower as txt1,class 
								from links 
								inner join text as text1 on text1.ID==source""")

In [48]:
inner_join.head(1)

[Row(source='9510123', target='9502114', txt1='an infinite number of potentials surrounding 2d black hole', class='1')]

In [53]:
#create features with word count on source								
def transform(row):
	data=row.asDict()#Rows are immutable; We convert them to dictionaries
	if data['txt1']==None : data['txt1']='' #We want o avoid any null issues
	#get unique words
	text1=set([x.lower() for x in  data['txt1'].encode('utf-8').split()])
	#create a new column with an array of number to be the features
	data["features"]=[len(text1)]#only one feature
	#keep only features and class
	ret={}
	ret["features"]=data["features"]
	ret["class"]=float(data["class"])
	#convert the dictionary back to a Row
	newRow = Row(*ret.keys()) #a. the Row object specification (column names)
	newRow = newRow(*ret.values())#b. the corresponding column values
	return newRow

In [54]:
data=inner_join.rdd.map(transform).toDF()	
print("New dataframe with map on RDD")
print(data.head(5))
print("################")	
#models use sparse arrays	

New dataframe with map on RDD
[Row(features=[9], class=1.0), Row(features=[8], class=1.0), Row(features=[6], class=0.0), Row(features=[9], class=0.0), Row(features=[5], class=0.0)]
################


In [59]:
#Lets apply a function directly on the dataframe
#UDF
#applying functions to a DATAFRAME requires the "SQL" logic of 
#User Defined Functions (UDF)
#as an example: convert features to sparse array
#1. define what data type your UDF returns VectorUDT : UDF SCHEMA
custom_udf_schema = VectorUDT()
#2. define function 
def to_sparse_(v):
		import numpy as np
		if isinstance(v, SparseVector):
			return v
		vs = np.array(v)
		nonzero = np.nonzero(vs)[0]
		return SparseVector(len(v), nonzero, vs[nonzero])
#3. create a udf from that function and the schema
to_sparse = udf(to_sparse_,custom_udf_schema)
#4. apply UDF to DF and create new column

In [60]:
data= data.withColumn('feats',to_sparse(data.features))

print("new sparse array column")
print(data.head(5))
print("################")

new sparse array column


[Row(features=[9], class=1.0, feats=SparseVector(1, {0: 9.0})), Row(features=[8], class=1.0, feats=SparseVector(1, {0: 8.0})), Row(features=[6], class=0.0, feats=SparseVector(1, {0: 6.0})), Row(features=[9], class=0.0, feats=SparseVector(1, {0: 9.0})), Row(features=[5], class=0.0, feats=SparseVector(1, {0: 5.0}))]
################


In [33]:
inner_join.head(10)

[Row(source='9510123', target='9502114', txt1='an infinite number of potentials surrounding 2d black hole', class='1'),
 Row(source='9707075', target='9604178', txt1='d 6 n 1 string vacua and duality', class='1'),
 Row(source='9312155', target='9506142', txt1='conformal field theory and hyperbolic geometry', class='0'),
 Row(source='9911255', target='302165', txt1='comparing instanton contributions with exact results in n 2', class='0'),
 Row(source='9701033', target='209076', txt1='quantum gravitational measure for three-geometries', class='0'),
 Row(source='9710020', target='9709228', txt1='n 4 4 2d supersymmetric gauge theory and brane configuration', class='1'),
 Row(source='9901042', target='9510135', txt1='k-theory reality and orientifolds', class='1'),
 Row(source='209146', target='9502077', txt1='logarithmic correlation functions in liouville field theory', class='0'),
 Row(source='9705079', target='9702201', txt1='hermitian d-brane solutions', class='1'),
 Row(source='3016', t